# Portfolio selection

https://docs.opvious.io/guides/examples/portfolio-optimization

In [ ]:
%pip install opvious[aio] yfinance

In [1]:
import opvious
import pandas as pd
import yfinance as yf

client = opvious.Client.from_environment()

In [2]:
tickers_df = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
tickers_df.head()

,Symbol,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3M,reports,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1976-08-09,66740,1902
1,AOS,A. O. Smith,reports,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888
3,ABBV,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989


In [12]:
values_df = yf.download(tickers=list(tickers_df['Symbol'])[:10], start='2022-1-1', interval='1mo')['Adj Close']
returns_df = values_df.head(100).dropna(axis=1).pct_change().dropna(axis=0, how='all')
returns_df.head()

[*********************100%***********************]  10 of 10 completed


,AAP,ABBV,ABT,ACN,ADBE,ADM,ADP,AOS,ATVI,MMM
Date,,,,,,,,,,
2022-02-01,-0.116755,0.090682,-0.050326,-0.103911,-0.124686,0.046000,-0.008391,-0.099266,0.031515,-0.104626
2022-03-01,0.012128,0.097043,-0.018737,0.067116,-0.025787,0.156653,0.112992,-0.068387,-0.017055,0.011229
2022-04-01,-0.028394,-0.093949,-0.041061,-0.109332,-0.130964,-0.007755,-0.036356,-0.085459,-0.056298,-0.031300
2022-05-01,-0.048941,0.011863,0.039015,-0.003367,0.051850,0.014069,0.021817,0.033525,0.036282,0.035155
2022-06-01,-0.088328,0.039289,-0.075004,-0.069725,-0.121062,-0.141624,-0.057863,-0.090486,-0.000257,-0.124404


In [22]:
inputs = await client.assemble_inputs(
    formulation_name='portfolio-selection',
    parameters={
        'assetCovariance': returns_df.cov().stack(),
        'expectedAssetReturn': returns_df.mean(),
        'minimumReturn': 0.02,
        'assetGroup': {},
        'minGroupAllocation': {},

    },
    infer_dimensions=True
)

In [23]:
attempt = await client.start_attempt(inputs)
outcome = await client.wait_for_outcome(attempt)

Tracking attempt... [url=https://hub.beta.opvious.io/attempts/57f7e7c6-9b2b-4ad1-98af-004e9a8fa79e]
Attempt is running... [elapsed=206 milliseconds]
Attempt is optimal. [elapsed=557 milliseconds, objective=0.003425327414321916]


In [24]:
allocation = await client.fetch_variable(attempt, 'allocation')
allocation.reset_index(names=['ticker']).join(
    returns_df.agg(['mean', 'var']).T,
    on='ticker',
    validate='one_to_one'
).sort_values(by=['value'], ascending=False)

,ticker,value,dual_value,mean,var
4,ABBV,0.862315,0.000000,0.020658,0.004262
3,ADP,0.090258,0.000000,0.015684,0.005681
0,ADM,0.047427,0.000000,0.016244,0.009291
1,ATVI,0.000000,0.015404,-0.000636,0.000966
2,ABT,0.000000,0.023491,-0.007751,0.002221
5,AOS,0.000000,0.028031,-0.011564,0.009924
6,ACN,0.000000,0.034752,-0.017248,0.006806
7,ADBE,0.000000,0.048237,-0.029867,0.013635
8,MMM,0.000000,0.030844,-0.014666,0.007240
9,AAP,0.000000,0.038132,-0.022739,0.012742
